In [98]:
import numpy as np
import pandas as pd

np.random.seed(7)

In [99]:
# import snscrape.modules.twitter as tw

# f = open('em.txt','w',encoding='utf-8')

# #Save Twitter data to external file
# #Possible outputs: url, date, content, id, username, outlinks, outlinksss, tcooutlinks, tcooutlinksss
# for tweet in tw.TwitterSearchScraper(query="(from:elonmusk) since:2020-08-01 until:2020-11-01").get_items():
#     date_str = tweet.date.strftime("%Y-%m-%d %H:%M:%S%z")
#     date_str = date_str[:-2] + ":" + date_str[-2:]
#     #f.write(date_str + "|" + tweet.content + "\n")
#     f.write(date_str + "|" + tweet.rawContent + "\n")
# f.close()

In [100]:
from datetime import datetime as dt
import pytz

#Read Twitter data into Python
em = []
dates = []
f = open("em.txt", "r", encoding="utf-8")
for l in f:
    line = l.split("|")
    date_str = line[0]#+"+00:00"
    try:
        date_time = dt.fromisoformat(date_str)
        date_time = date_time.astimezone(pytz.timezone("US/Eastern"))
        line[0] = date_time
        line[1] = line[1][:-1]
        em.append(line)
        dates.append(date_time.date())
    except:
        em[-1][1] += " "+l[:-1]
f.close()

em = pd.DataFrame(data=em , columns=['Time','Tweet'])
em['Date'] = dates
#em['Date'] = em['Date'].astype(str)
print(em)

                         Time  \
0   2020-10-31 19:34:19-04:00   
1   2020-10-31 18:09:40-04:00   
2   2020-10-31 18:04:30-04:00   
3   2020-10-31 18:00:41-04:00   
4   2020-10-31 17:59:44-04:00   
..                        ...   
760 2020-08-02 00:49:11-04:00   
761 2020-08-01 04:15:54-04:00   
762 2020-08-01 02:47:18-04:00   
763 2020-07-31 23:57:37-04:00   
764 2020-07-31 23:51:54-04:00   

                                                 Tweet        Date  
0                       @RGVaerialphotos Lord of the …  2020-10-31  
1    @TGMetsFan98 @KlotzAdam @NASASpaceflight @Erda...  2020-10-31  
2      @flcnhvy @NASASpaceflight @Erdayastronaut Minor  2020-10-31  
3    @NASASpaceflight @Erdayastronaut But, a RUD ri...  2020-10-31  
4    @NASASpaceflight @Erdayastronaut Stable, contr...  2020-10-31  
..                                                 ...         ...  
760  @meier1028 @SpaceX @Space_Station @AstroBehnke...  2020-08-02  
761  @engineeringvids The simplest solution is not 

In [101]:
# Pre-process for Latent Dirichlet Allocation
from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

# Initialize regex tokenizer
tokenizer = RegexpTokenizer(r'\w+') # Tokenizes by word

# Vectorize in one of 2 ways:
# Term frequency:
tf = CountVectorizer(lowercase=True,stop_words='english',ngram_range=(1,1),tokenizer=tokenizer.tokenize)
tf_data = tf.fit_transform(em.Tweet.tolist())
tf_feature_names = tf.get_feature_names_out()

# Term frequency-inverse document frequency
tfidf = TfidfVectorizer(lowercase=True,stop_words='english',ngram_range=(1,1),tokenizer=tokenizer.tokenize)
tfidf_data = tfidf.fit_transform(em.Tweet.tolist())
tfidf_feature_names = tfidf.get_feature_names_out()

In [102]:
# Run Latent Dirichlet Allocation
from sklearn.decomposition import LatentDirichletAllocation

K = 5 # Number of topics

# Try with term frequency:
LDA_TF = LatentDirichletAllocation(n_components=K)
LDA_TF_Matrix = LDA_TF.fit_transform(tf_data) # Fits and transforms the dataset
LDA_TF_Components = LDA_TF.components_ # Get the components

# Try with term frequency-inverse document frequency:
LDA_TFIDF = LatentDirichletAllocation(n_components=K)
LDA_TFIDF_Matrix = LDA_TFIDF.fit_transform(tfidf_data) # Fits and transforms the dataset
LDA_TFIDF_Components = LDA_TFIDF.components_ # Get the components

In [103]:
# Displaying Topics
def display_topics(components, feature_names, no_top_words):
    for index, topic in enumerate(components):
        top_terms_list = [feature_names[i] for i in topic.argsort()[:-no_top_words-1:-1]]
        print("Topic "+str(index+1)+": ",top_terms_list)

no_top_words = 10
print("Term Frequency")
display_topics(LDA_TF_Components , tf_feature_names , no_top_words)

print(" ")
print("Term Frequency-Inverse Document Frequency")
display_topics(LDA_TFIDF_Components , tfidf_feature_names , no_top_words)

Term Frequency
Topic 1:  ['ppathole', 'tesla', 'flcnhvy', 's', 'yes', 'teslaownerssv', 'spacex', 'amp', 'evafoxu', 'true']
Topic 2:  ['amp', 'erdayastronaut', 'flcnhvy', 'spacex', 'ppathole', 's', 'wholemarsblog', 'yes', 'tesla', 'kristennetten']
Topic 3:  ['t', 'https', 'tesla', 'yes', 'amp', 'erdayastronaut', 'wholemarsblog', 'spacex', 'teslarati', 'waitbutwhy']
Topic 4:  ['t', 'https', 's', 'tesla', 'spacex', 'amp', 'ppathole', 'great', 'teslaownerssv', 'wholemarsblog']
Topic 5:  ['amp', 'nasaspaceflight', 'spacex', 'erdayastronaut', 's', 'tesla', 'production', 'high', 'marcushousegame', 'flcnhvy']
 
Term Frequency-Inverse Document Frequency
Topic 1:  ['erdayastronaut', 'amp', 'spacex', 't', 'cleantechnica', 'ppathole', 'https', 'need', 'flcnhvy', 's']
Topic 2:  ['ppathole', 'tesla', 'amp', 'teslaownerssv', 'soon', 't', 'wholemarsblog', 'coming', 'https', 'sure']
Topic 3:  ['t', 'https', 'evafoxu', 'great', 'spacex', 's', 'pretty', 'years', 'teslarati', 'neuralink']
Topic 4:  ['yes'

In [104]:
# Classifying the Documents
# Let's just look at TF-IDF
print(LDA_TFIDF_Matrix)

label = np.argmax(LDA_TFIDF_Matrix , axis=1)
em['Cluster'] = label
print(em)

[[0.6676496  0.0829821  0.08298097 0.08298015 0.08340717]
 [0.0457915  0.04477599 0.04428458 0.04458284 0.82056508]
 [0.06942209 0.0685405  0.06868892 0.06958301 0.72376548]
 ...
 [0.2        0.2        0.2        0.2        0.2       ]
 [0.77798866 0.05642555 0.05515358 0.05520521 0.05522701]
 [0.05370261 0.05383614 0.78441895 0.05372717 0.05431514]]
                         Time  \
0   2020-10-31 19:34:19-04:00   
1   2020-10-31 18:09:40-04:00   
2   2020-10-31 18:04:30-04:00   
3   2020-10-31 18:00:41-04:00   
4   2020-10-31 17:59:44-04:00   
..                        ...   
760 2020-08-02 00:49:11-04:00   
761 2020-08-01 04:15:54-04:00   
762 2020-08-01 02:47:18-04:00   
763 2020-07-31 23:57:37-04:00   
764 2020-07-31 23:51:54-04:00   

                                                 Tweet        Date  Cluster  
0                       @RGVaerialphotos Lord of the …  2020-10-31        0  
1    @TGMetsFan98 @KlotzAdam @NASASpaceflight @Erda...  2020-10-31        4  
2      @flcnhvy

In [105]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

#The compound score is computed by summing the valence scores of each word in the lexicon, 
#adjusted according to the rules, and then normalized to be between -1 (most extreme negative) 
#and +1 (most extreme positive). This is the most useful metric if you want a single unidimensional 
#measure of sentiment for a given sentence. Calling it a 'normalized, weighted composite score' is accurate.
#It is also useful for researchers who would like to set standardized thresholds for classifying sentences 
#as either positive, neutral, or negative. Typical threshold values (used in the literature cited on this page) are:
#    positive sentiment: compound score >= 0.05
#    neutral sentiment: (compound score > -0.05) and (compound score < 0.05)
#    negative sentiment: compound score <= -0.05
#The pos, neu, and neg scores are ratios for proportions of text that fall in each category (so these should all 
#add up to be 1... or close to it with float operation). These are the most useful metrics if you want 
#multidimensional measures of sentiment for a given sentence.

sentiment = []
analyzer = SentimentIntensityAnalyzer()
for tweet in em.Tweet:
    vs = analyzer.polarity_scores(tweet)
    sentiment.append(vs["compound"])
    
em['Sentiment'] = sentiment
print(em)

                         Time  \
0   2020-10-31 19:34:19-04:00   
1   2020-10-31 18:09:40-04:00   
2   2020-10-31 18:04:30-04:00   
3   2020-10-31 18:00:41-04:00   
4   2020-10-31 17:59:44-04:00   
..                        ...   
760 2020-08-02 00:49:11-04:00   
761 2020-08-01 04:15:54-04:00   
762 2020-08-01 02:47:18-04:00   
763 2020-07-31 23:57:37-04:00   
764 2020-07-31 23:51:54-04:00   

                                                 Tweet        Date  Cluster  \
0                       @RGVaerialphotos Lord of the …  2020-10-31        0   
1    @TGMetsFan98 @KlotzAdam @NASASpaceflight @Erda...  2020-10-31        4   
2      @flcnhvy @NASASpaceflight @Erdayastronaut Minor  2020-10-31        4   
3    @NASASpaceflight @Erdayastronaut But, a RUD ri...  2020-10-31        4   
4    @NASASpaceflight @Erdayastronaut Stable, contr...  2020-10-31        4   
..                                                 ...         ...      ...   
760  @meier1028 @SpaceX @Space_Station @AstroBehnk

In [106]:
import yfinance
from datetime import datetime

#Load daily Tesla Adjusted Close data and compute log returns
tesla = yfinance.download("TSLA",datetime(2020,8,1),datetime(2020,10,1)) 
tesla['log_ret'] = np.log(tesla['Adj Close'].shift(-1)) - np.log(tesla['Adj Close'])

#Construct average sentiment on each day for each topic
for k in range(K):
    prev_d = datetime(2020,1,1).date()
    avg_sentiment = []
    for date in tesla.index.values:
        d = pd.to_datetime(date).date()
        avg = np.mean(em.loc[(em.Date <= d) & (em.Date > prev_d) & (em.Cluster == k)].Sentiment)
        if np.isnan(avg):
            avg_sentiment.append(0)
        else:
            avg_sentiment.append(avg)
        prev_d = d
    tesla['sentiment '+str(k+1)] = avg_sentiment

#Drop final row
tesla = tesla.iloc[:-1,]
print(tesla)

[*********************100%***********************]  1 of 1 completed
                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2020-08-03   96.613335  100.653999   96.292000   99.000000   99.000000   
2020-08-04   99.667336  101.827332   97.466667   99.133331   99.133331   
2020-08-05   99.532669   99.989334   97.887337   99.001335   99.001335   
2020-08-06   99.388664  101.153999   98.484001   99.305336   99.305336   
2020-08-07   99.969330   99.983330   94.334000   96.847336   96.847336   
2020-08-10   96.533333   97.166664   92.389336   94.571335   94.571335   
2020-08-11   93.066666   94.666664   91.000000   91.625999   91.625999   
2020-08-12   98.000000  105.666664   95.666664  103.650665  103.650665   
2020-08-13  107.400002  110.078667  104.484001  108.066666  108.066666   
2020-08-14  110.999336  111.253334  108.442665  110.047333  110.047333   
2020-08-17  111.800003  123.057335  111.522

In [107]:
#Find correlation between Tesla Returns and Elon Musk's Twitter account
#Recall that we had low correlation when looking at all tweets
for k in range(K):
    print('Correlation with Topic '+str(k+1)+': '+str(tesla['log_ret'].corr(tesla['sentiment '+str(k+1)])))

Correlation with Topic 1: 0.34416126831827
Correlation with Topic 2: -0.2078878186430723
Correlation with Topic 3: 0.00023618005818470378
Correlation with Topic 4: -0.17769576983810267
Correlation with Topic 5: 0.24683650182448783


In [1]:
# healthcare etf and 10 healthcare stock tickers
tickers = [
    'XLV',
    'JNJ',
    'PFE',
    'MRK',
    'UNH',
    'ABT',
    'ABBV',
    'AMGN',
    'MDT',
    'DHR',
    'CVS',
    'MMM',
    'GE',
    'BA',
    'CAT'
]

15